# Team Model

In [ ]:
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from airsenal.framework.bpl_interface import DEFAULT_TEAM_EPSILON, get_fitted_team_model
from airsenal.framework.schema import session
from airsenal.framework.season import CURRENT_SEASON, CURRENT_TEAMS
from airsenal.framework.utils import NEXT_GAMEWEEK, get_fixtures_for_gameweek

## Fit Model

In [ ]:
model_team = get_fitted_team_model(CURRENT_SEASON, NEXT_GAMEWEEK, session)

In [ ]:
# extract indices of current premier league teams
# val-1 because 1-indexed in model but 0-indexed in python
current_idx = {
    team: idx for idx, team in enumerate(model_team.teams) if team in CURRENT_TEAMS
}

## Match Outcome Predictions

In [ ]:
fixtures = get_fixtures_for_gameweek(NEXT_GAMEWEEK, CURRENT_SEASON)

nrow = int(np.ceil(len(fixtures) / 2))
fig, ax = plt.subplots(nrow, 2, figsize=(8, 2.2 * nrow), sharey=True)
ax = ax.flatten()
max_goals = 10

for i, f in enumerate(fixtures):
    prob_score_h = [
        model_team.predict_score_n_proba(n, f.home_team, f.away_team)[0]
        for n in range(max_goals)
    ]
    exp_h = sum([n * prob_score_h[n] for n in range(max_goals)]) / sum(prob_score_h)

    prob_score_a = [
        model_team.predict_score_n_proba(n, f.away_team, f.home_team, home=False)[0]
        for n in range(max_goals)
    ]
    exp_a = sum([n * prob_score_a[n] for n in range(max_goals)]) / sum(prob_score_a)

    outcome_proba = model_team.predict_outcome_proba(f.home_team, f.away_team)
    ax[i].bar(
        range(max_goals),
        prob_score_h,
        facecolor="none",
        edgecolor="b",
        label=f.home_team,
    )
    ax[i].bar(
        range(max_goals),
        prob_score_a,
        facecolor="none",
        edgecolor="r",
        label=f.away_team,
    )

    ax[i].set_xlim([-1, max_goals])
    bs = r"$\bf{"
    be = r"}$"
    ax[i].set_title(
        f"{bs}{f.home_team}{be} {bs}vs.{be} {bs}{f.away_team}{be}\n"
        f"{exp_h:.2f} goals vs. {exp_a:.2f} goals\n"
        f"Home win: {outcome_proba['home_win'][0]:.2f}, "
        f"Draw: {outcome_proba['draw'][0]:.2f}, "
        f"Away win: {outcome_proba['away_win'][0]:.2f}",
        fontsize=10,
    )
    ax[i].set_ylabel("Probability")
    ax[i].set_xlabel("Goals")
    ax[i].legend()

fig.suptitle(
    f"Predicted Match Outcomes for GW{NEXT_GAMEWEEK}, Season {CURRENT_SEASON}",
    weight="bold",
)
fig.tight_layout()

## Team Strengths / Model Parameters

In [ ]:
a_mean = model_team.attack.mean(axis=0)
b_mean = model_team.defence.mean(axis=0)

a_conf = np.abs(
    np.quantile(model_team.attack, [0.5 - 1 / 3, 0.5 + 1 / 3], axis=0) - a_mean
)
b_conf = np.abs(
    np.quantile(model_team.defence, [0.5 - 1 / 3, 0.5 + 1 / 3], axis=0) - b_mean
)

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.set_aspect("equal")
select_idx = jnp.array(list(current_idx.values()), dtype=int)
plt.errorbar(
    a_mean[select_idx],
    b_mean[select_idx],
    xerr=a_conf[:, select_idx],
    yerr=b_conf[:, select_idx],
    marker="o",
    markersize=10,
    linestyle="",
    linewidth=0.5,
)
plt.xlabel("attack", fontsize=14)
plt.ylabel("defence", fontsize=14)

for team, idx in current_idx.items():
    ax.annotate(team, (a_mean[idx] - 0.03, b_mean[idx] + 0.02), fontsize=12)

In [ ]:
plt.figure(figsize=(10, 5))
sns.violinplot(model_team.attack[:, list(current_idx.values())])
plt.xticks(
    ticks=np.arange(len(current_idx)), labels=list(current_idx.keys()), rotation=90
)
plt.title("Attack")
plt.axhline(0, color="k")

In [ ]:
plt.figure(figsize=(10, 5))
sns.violinplot(model_team.defence[:, list(current_idx.values())])
plt.xticks(
    ticks=np.arange(len(current_idx)), labels=list(current_idx.keys()), rotation=90
)
plt.title("Defence")
plt.axhline(0, color="k")

In [ ]:
plt.figure(figsize=(10, 5))
sns.violinplot(model_team.home_advantage[:, list(current_idx.values())])
plt.xticks(
    ticks=np.arange(len(current_idx)), labels=list(current_idx.keys()), rotation=90
)
plt.title("Home Advantage")
plt.axhline(0, color="k")

In [ ]:
plt.figure()
for i in range(model_team.attack_coefficients.shape[1]):
    sns.kdeplot(model_team.attack_coefficients[:, i])
plt.title("Attack Coefficients (Covariates)")

plt.figure()
for i in range(model_team.defence_coefficients.shape[1]):
    sns.kdeplot(model_team.defence_coefficients[:, i])
plt.title("Defence Coefficients (Covariates)")

In [ ]:
beta_a_mean = model_team.attack_coefficients.mean(axis=0)
beta_b_mean = model_team.defence_coefficients.mean(axis=0)

beta_a_conf95 = np.abs(
    np.quantile(model_team.attack_coefficients, [0.025, 0.975], axis=0) - beta_a_mean
)
beta_b_conf95 = np.abs(
    np.quantile(model_team.defence_coefficients, [0.025, 0.975], axis=0) - beta_b_mean
)
beta_a_conf80 = np.abs(
    np.quantile(model_team.attack_coefficients, [0.1, 0.9], axis=0) - beta_a_mean
)
beta_b_conf80 = np.abs(
    np.quantile(model_team.defence_coefficients, [0.1, 0.9], axis=0) - beta_b_mean
)

fig, ax = plt.subplots(1, 1, figsize=(7, 7))
ax.set_aspect("equal")
plt.errorbar(
    beta_a_mean,
    beta_b_mean,
    xerr=beta_a_conf80,
    yerr=beta_b_conf80,
    marker="o",
    markersize=10,
    linestyle="",
    linewidth=0.5,
)
plt.xlabel("beta_a", fontsize=14)
plt.ylabel("beta_b", fontsize=14)
plt.title("FIFA Ratings")

for idx, feat in enumerate(["att", "mid", "defn", "ovr"]):
    ax.annotate(feat, (beta_a_mean[idx] - 0.03, beta_b_mean[idx] + 0.02), fontsize=12)

xlim = ax.get_xlim()
ylim = ax.get_ylim()
plt.plot([0, 0], ylim, color="k", linewidth=0.75)
plt.plot(xlim, [0, 0], color="k", linewidth=0.75)
plt.xlim(xlim)
plt.ylim(ylim)

In [ ]:
sns.kdeplot(model_team.rho)
plt.title("rho")
print(f"Rho: {model_team.rho.mean():.2f}")

### Change vs. `epsilon=0`

In [ ]:
model_team = get_fitted_team_model(CURRENT_SEASON, NEXT_GAMEWEEK, session, epsilon=0)
select_idx = jnp.array(
    [list(model_team.teams).index(t) for t in CURRENT_TEAMS], dtype=int
)
a_mean_1 = model_team.attack.mean(axis=0)[select_idx]
b_mean_1 = model_team.defence.mean(axis=0)[select_idx]

model_team = get_fitted_team_model(CURRENT_SEASON, NEXT_GAMEWEEK, session)
select_idx = jnp.array(
    [list(model_team.teams).index(t) for t in CURRENT_TEAMS], dtype=int
)
a_mean_2 = model_team.attack.mean(axis=0)[select_idx]
b_mean_2 = model_team.defence.mean(axis=0)[select_idx]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
ax.set_aspect("equal")

for a_1, b_1, a_2, b_2 in zip(a_mean_1, b_mean_1, a_mean_2, b_mean_2, strict=True):
    plt.arrow(
        a_1,
        b_1,
        a_2 - a_1,
        b_2 - b_1,
        width=0.005,
        length_includes_head=True,
    )
plt.xlabel("attack", fontsize=14)
plt.ylabel("defence", fontsize=14)

for idx, team in enumerate(CURRENT_TEAMS):
    ax.annotate(team, (a_mean_2[idx] - 0.02, b_mean_2[idx] + 0.005), fontsize=12)

plt.title(f"epsilon={DEFAULT_TEAM_EPSILON} vs. epsilon=0")

### Change since start of season

In [ ]:
model_team = get_fitted_team_model(CURRENT_SEASON, 1, session)
select_idx = jnp.array(
    [list(model_team.teams).index(t) for t in CURRENT_TEAMS], dtype=int
)
a_mean_1 = model_team.attack.mean(axis=0)[select_idx]
b_mean_1 = model_team.defence.mean(axis=0)[select_idx]

model_team = get_fitted_team_model(CURRENT_SEASON, NEXT_GAMEWEEK, session)
select_idx = jnp.array(
    [list(model_team.teams).index(t) for t in CURRENT_TEAMS], dtype=int
)
a_mean_2 = model_team.attack.mean(axis=0)[select_idx]
b_mean_2 = model_team.defence.mean(axis=0)[select_idx]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
ax.set_aspect("equal")

for a_1, b_1, a_2, b_2 in zip(a_mean_1, b_mean_1, a_mean_2, b_mean_2, strict=True):
    plt.arrow(
        a_1,
        b_1,
        a_2 - a_1,
        b_2 - b_1,
        width=0.005,
        length_includes_head=True,
    )
plt.xlabel("attack", fontsize=14)
plt.ylabel("defence", fontsize=14)

for idx, team in enumerate(CURRENT_TEAMS):
    ax.annotate(team, (a_mean_2[idx] - 0.02, b_mean_2[idx] + 0.005), fontsize=12)

plt.title(f"GW{NEXT_GAMEWEEK} vs. GW1")
plt.show()

delta_a = a_mean_2 - a_mean_1
delta_b = b_mean_2 - b_mean_1

print("Attack strength change since start of season:")
display(pd.Series(delta_a, index=CURRENT_TEAMS).sort_values(ascending=False))

print("Defence strength change since start of season:")
display(pd.Series(delta_b, index=CURRENT_TEAMS).sort_values(ascending=False))

print("Overall magnitude of team strength change since start of season:")
surprise = np.sqrt(delta_a**2 + delta_b**2)
display(pd.Series(surprise, index=CURRENT_TEAMS).sort_values(ascending=False))

### Change since last gameweek 

In [ ]:
model_team = get_fitted_team_model(CURRENT_SEASON, NEXT_GAMEWEEK - 1, session)
select_idx = jnp.array(
    [list(model_team.teams).index(t) for t in CURRENT_TEAMS], dtype=int
)
a_mean_1 = model_team.attack.mean(axis=0)[select_idx]
b_mean_1 = model_team.defence.mean(axis=0)[select_idx]

model_team = get_fitted_team_model(CURRENT_SEASON, NEXT_GAMEWEEK, session)
select_idx = jnp.array(
    [list(model_team.teams).index(t) for t in CURRENT_TEAMS], dtype=int
)
a_mean_2 = model_team.attack.mean(axis=0)[select_idx]
b_mean_2 = model_team.defence.mean(axis=0)[select_idx]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
ax.set_aspect("equal")
plt.axhline(0, color="k", linewidth=0.5)
plt.axvline(0, color="k", linewidth=0.5)

delta_a = a_mean_2 - a_mean_1
delta_b = b_mean_2 - b_mean_1

for da, db, t in zip(delta_a, delta_b, CURRENT_TEAMS, strict=True):
    plt.plot(
        da,
        db,
        marker="none",
    )
    plt.text(
        da,
        db,
        t,
        horizontalalignment="center",
        verticalalignment="center",
        fontsize=10,
    )

plt.xlabel(r"$\Delta$(attack)", fontsize=11)
plt.ylabel(r"$\Delta$(defence)", fontsize=11)
plt.title(f"GW{NEXT_GAMEWEEK} vs. GW{NEXT_GAMEWEEK - 1}")
plt.show()

surprise = np.sqrt(delta_a**2 + delta_b**2)
print(
    f"Magnitude of team strength updates GW{NEXT_GAMEWEEK} vs. GW{NEXT_GAMEWEEK - 1}:"
)
pd.Series(surprise, index=CURRENT_TEAMS).sort_values(ascending=False)

### Changes during season (slow)

In [ ]:
a_mean = np.full((len(CURRENT_TEAMS), NEXT_GAMEWEEK), np.nan)
b_mean = np.full((len(CURRENT_TEAMS), NEXT_GAMEWEEK), np.nan)

for gw in range(1, NEXT_GAMEWEEK + 1):
    model_team = get_fitted_team_model(CURRENT_SEASON, gw, session)
    select_idx = jnp.array(
        [list(model_team.teams).index(t) for t in CURRENT_TEAMS], dtype=int
    )
    a_mean[:, gw - 1] = model_team.attack.mean(axis=0)[select_idx]
    b_mean[:, gw - 1] = model_team.defence.mean(axis=0)[select_idx]

In [ ]:
fig = plt.figure(figsize=(8, 8))
for a_team, b_team, team in zip(a_mean, b_mean, CURRENT_TEAMS, strict=True):
    plt.plot(a_team, b_team, marker="none", label=team)
    plt.annotate(team, (a_team[-1] - 0.02, b_team[-1] + 0.01), fontsize=12)
plt.xlabel("Attack")
plt.ylabel("Defence")
plt.axis("equal")
fig.legend(
    loc="outside lower center", ncol=len(CURRENT_TEAMS) / 4, bbox_to_anchor=(0.5, -0.07)
)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 6), sharex=True)
for idx, team in enumerate(CURRENT_TEAMS):
    ax[0].plot(
        np.arange(1, NEXT_GAMEWEEK + 1),
        a_mean[idx],
        marker="o",
        label=team,
    )
    ax[1].plot(
        np.arange(1, NEXT_GAMEWEEK + 1),
        b_mean[idx],
        marker="o",
    )
ax[0].set_xlabel("Gameweek")
ax[1].set_xlabel("Gameweek")
ax[0].set_title("Attack")
ax[1].set_title("Defence")
fig.legend(
    loc="outside lower center", ncol=len(CURRENT_TEAMS) / 2, bbox_to_anchor=(0.5, -0.15)
)